In [ ]:
'''
自作コード
'''
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import datetime
import sys
import os
import re
from robobrowser import RoboBrowser
import re
import pandas as pd 
# RoboBrowserオブジェクトを作成する。
browser = RoboBrowser(
    parser='html.parser',  # Beautiful Soupで使用するパーサーを指定する。
    # Cookieが使用できないと表示されてログインできない問題を回避するため、
    # 通常のブラウザーのUser-Agent（ここではFirefoxのもの）を使う。
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')


def main():
    df2=pd.DataFrame()

    # 注文履歴のページを開く。
    print('Navigating...', file=sys.stderr)
 
    # browser.open('https://www.amazon.co.jp/gp/product/4873116554/ref=s9_acsd_simh_hd_bw_b1x4K_c_x_3_w?pf_rd_m=AN1VRQENFRJN5&pf_rd_s=merchandised-search-4&pf_rd_r=GGPKJ52QZZAWFHSKR0RF&pf_rd_t=101&pf_rd_p=84c28d47-4e2f-5adb-b380-a7b7809ae8ee&pf_rd_i=465392')
#     browser.open('https://www.amazon.co.jp/Python%E3%81%AB%E3%82%88%E3%82%8B%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90%E5%85%A5%E9%96%80-%E2%80%95NumPy%E3%80%81pandas%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E3%83%87%E3%83%BC%E3%82%BF%E5%87%A6%E7%90%86-Wes-McKinney/product-reviews/4873116554/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1')
    browser.open('https://www.amazon.co.jp/%E3%83%90%E3%82%AB%E3%81%AE%E5%A3%81-%E6%96%B0%E6%BD%AE%E6%96%B0%E6%9B%B8-%E9%A4%8A%E8%80%81-%E5%AD%9F%E5%8F%B8/product-reviews/4106100037/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')


    # ページャーをたどる。
    while True:
        df=pd.DataFrame()
#         assert 'Python' in browser.parsed.title.string  # 注文履歴画面が表示されていることを確認する。
        print(browser.parsed.title.string)
        # print(browser.parsed.prettify())
        df=print_order_history()
        
        df2=pd.concat([df2,df],axis=0)

        link_to_next = browser.get_link('次へ')  # 「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            return(df2)

        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next)  # 「次へ」というリンクをたどる。

def print_order_history():

    # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる。
    for i,line_item in enumerate(browser.select('.a-section.celwidget')):
        review_rank=line_item.select('.a-icon-alt')
        review_date=line_item.select('.review-date')
        review_text=line_item.select('.review-text')
#         review_vote=line_item.select('.cr-vote-text')
        
        if i>0:
            review_rank=[x.get_text() for x in review_rank]
            review_rank=[float(re.sub('5つ星のうち','',x)) for x in review_rank]
#             review_vote=[x.get_text() for x in review_vote]
#             review_vote=[float(re.sub('人のお客様がこれが役に立ったと考えています','',x)) for x in review_vote]
            review_text=[x.get_text() for x in review_text]
            
            review_len_text=[len(x) for x in review_text]
            date_formated=[datetime.datetime.strptime(x.get_text(),'%Y年%m月%d日') for x in review_date]
            df=pd.DataFrame({
				'date':date_formated,
				'rank':review_rank,
# 				'vote':review_vote,
				'len_text':review_len_text
			})	
            
            return(df)

In [ ]:
df3=main()

In [ ]:
def create_ts_rate(df3):
    df2=df3.copy()
    df2=df2.sort_values('date').reset_index()
    df2['cumsum_rate']=df2['rank'].cumsum()
    df2['ts_rate']=df2['cumsum_rate']/list(df2.index+1)
    df2.index=pd.to_datetime(df2.date)
    df2=df2.drop('date',axis=1)
    monthly=df2.groupby(pd.Grouper(level=0,freq='M')).mean()
    monthly=monthly.fillna(method='bfill')    
    return(monthly['ts_rate'])

In [ ]:
df4=create_ts_rate(df3)